In [9]:
import tensorflow as tf
import numpy as np

tf.metrics.accuracy(  
    labels,  
    predictions,  
    weights=None,  
    metrics_collections=None,  
    updates_collections=None,  
    name=None  
)  


In [2]:
tf.reset_default_graph()
y = tf.placeholder(dtype=tf.int32, shape=[10])
y_pred = tf.placeholder(dtype=tf.int32, shape=[10])
acc, acc_update_op = tf.metrics.accuracy(y, y_pred)

In [5]:
tf.get_collection(tf.GraphKeys.METRIC_VARIABLES)

[<tf.Variable 'accuracy/total:0' shape=() dtype=float32_ref>,
 <tf.Variable 'accuracy/count:0' shape=() dtype=float32_ref>]

In [6]:
tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES)

[<tf.Variable 'accuracy/total:0' shape=() dtype=float32_ref>,
 <tf.Variable 'accuracy/count:0' shape=() dtype=float32_ref>]

In [7]:
acc_init_op = tf.variables_initializer(tf.get_collection(tf.GraphKeys.METRIC_VARIABLES))

In [8]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(acc_init_op)

# Experiment 1

In [11]:
sess.run(acc_update_op, feed_dict={y : np.zeros(10, dtype=np.int32), y_pred : np.zeros(10, dtype=np.int32)})

1.0

In [12]:
sess.run(acc_update_op, feed_dict={y : np.zeros(10, dtype=np.int32), y_pred : np.ones(10, dtype=np.int32)})

0.5

In [13]:
sess.run(acc_update_op, feed_dict={y : np.int32([1,1,1,1,1,0, 0, 0, 0,1]), y_pred : np.ones(10, dtype=np.int32)})

0.53333336

In [14]:
sess.run(acc)

0.53333336

# Experiment 2

In [38]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(acc_init_op)

In [39]:
sess.run([y, y_pred], feed_dict={y : np.zeros(10, dtype=np.int32), y_pred : np.zeros(10, dtype=np.int32)})

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)]

In [40]:
sess.run([y, y_pred], feed_dict={y : np.zeros(10, dtype=np.int32), y_pred : np.ones(10, dtype=np.int32)})

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)]

In [41]:
sess.run([y, y_pred], feed_dict={y : np.int32([1,1,1,1,1,0, 0, 0, 0,1]), y_pred : np.ones(10, dtype=np.int32)})

[array([1, 1, 1, 1, 1, 0, 0, 0, 0, 1], dtype=int32),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)]

In [42]:
sess.run(acc)

0.0

# Custom metric

"""Create variable in `GraphKeys.(LOCAL|METRIC_VARIABLES`) collections."""  
metric_variable(shape, dtype, validate_shape=True, name=None)  

In [45]:
from tensorflow.python.ops import metrics_impl
def count_zeros_metric(values, metrics_collections=None, updates_collections=None, name=None):
    with tf.variable_scope(name, 'count_zeros', (values,)):
        
        # metric variables should be created using this function
        count = metrics_impl.metric_variable([], dtype=tf.int32, validate_shape=True, name='count')
        
        # main part of work
        zeros_in_predictions = tf.reduce_sum(tf.cast(tf.equal(values, 0), dtype=tf.int32))
        
        # change of values -> change of counter
        with tf.control_dependencies([values]):
            count_update_op = tf.assign_add(count, zeros_in_predictions)
        
        # add to custom collections
        if metrics_collections:
            tf.add_to_collection(metrics_collections, count)
            
        if updates_collections:
            tf.add_to_collection(updates_collections, count_update_op)
        return count, count_update_op

# Try to repeat experiments with accuracy 

In [46]:
count, count_update_op = count_zeros_metric(y_pred)

In [47]:
count_init_op = tf.variables_initializer(tf.get_collection(tf.GraphKeys.METRIC_VARIABLES))

In [49]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(count_init_op)

In [50]:
sess.run(count_update_op, feed_dict={y : np.zeros(10, dtype=np.int32), y_pred : np.zeros(10, dtype=np.int32)})

10

In [51]:
sess.run(count_update_op, feed_dict={y : np.zeros(10, dtype=np.int32), y_pred : np.ones(10, dtype=np.int32)})

10

In [52]:
sess.run(count_update_op, feed_dict={y_pred : np.int32([1,1,1,1,1,0, 0, 0, 0,1])})

14

In [53]:
sess.run(count)

14